In [30]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    TODO write the description
'''

datadir = "/datasets/eurora_data/db1/"
datadir = "/datasets/eurora_data/db/"
infile_jobs_to_nodes = datadir + "job_nodes.csv"


infile_nodes = datadir + "nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
infile_jobs = datadir + "CPUs/" + interval_comment + "/jobs_cleaned"

## 3 settings Alina
#interval_comment = "Alina"
#start_time = pd.to_datetime('2014-06-30')
#end_time = pd.to_datetime('2014-11-01')



Along the nodes we have three differentIntel Xeon E5 processors stepping: nodes 1-16 & 25-32 havea maximum frequency of 2.1GHz (CPUs-2100), nodes 17-24 have a maximum frequency of 2.2GHz (CPUs-2200), andnodes 33-64 have a maximum frequency of 3.1GHz (CPUs-3100).

In [22]:
nodes_data = pd.read_csv(infile_nodes, index_col=0)
# nodes_data

### Read job2nodes data

In [23]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records


#### solve the problem of jobs runned on node 129

In [24]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129]
print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data now contains 469020 records


#### solve the problem of duplicates job_id_string, node_id
this is probably caused by the concurrency writing of the database 

In [25]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data now contains 444610 records


### Read jobs data

drop the jobs out of interval

In [26]:
jobs_whole = pd.read_csv(infile_jobs + ".csv", index_col=0)
print("jobs_whole_data contains {} records".format(jobs_whole.shape[0]))

jobs_whole_data contains 88537 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### merge

In [28]:
merged_jobs_to_nodes_long = pd.merge(jobs_to_nodes_whole_data, jobs_whole, on='job_id_string')
print(merged_jobs_to_nodes_long.shape[0])

94962


#### drop the unused columns

In [13]:
### I will probably create a new dataframe "job_id_string: real_power"
# add two new column: real_power and entire_node
# completed_jobs_data_interval['entire_node'] = pd.Series()
# completed_jobs_data_interval['real_power'] = pd.Series(0)
# completed_jobs_data_interval.head(1)

### compute the real power consumption - long jobs (4h)

In [14]:
### remove to continue from a specific job

#long_jobs = long_jobs.drop(['real_pow', 'runned_alone', 'real_pow_quality'], axis=1)

jobs_whole['real_pow'] = 0.0
jobs_whole['runned_alone'] = True
jobs_whole['real_pow_quality'] = 0.0

jobs_whole['n_2_1'] = 0
jobs_whole['n_2_2'] = 0
jobs_whole['n_3_1'] = 0

In [15]:
#print(real_pow.shape[0])
#print(runned_alone.shape[0])
#print(quality.shape[0])

#large_jobs_tail = large_jobs[large_jobs.index > 390345]

In [25]:
#large_jobs_tail.shape

In [ ]:
i = 0
outfile = datadir + 'CPUs/' + interval_comment + "/" + interval_comment + "_long_jobs_real_pow.csv"
for index, row in jobs_whole.iterrows():
#for index, row in large_jobs_tail.iterrows():
    print()
    print("{}/{}".format(i, jobs_whole.shape[0]))

    #real_pow[i], runned_alone[i], quality[i] = calculate_job_consumption(row['job_id_string'])
    real_pow_val, runned_alone_val, quality_val, job_n_2_1, job_n_2_2, job_n_3_1 = calculate_job_consumption_2(row['job_id_string'])
    jobs_whole.at[index,'real_pow'] = real_pow_val
    jobs_whole.at[index,'runned_alone'] = runned_alone_val
    jobs_whole.at[index,'real_pow_quality'] = quality_val
    
    jobs_whole.at[index,'n_2_1'] = job_n_2_1
    jobs_whole.at[index,'n_2_2'] = job_n_2_2
    jobs_whole.at[index,'n_3_1'] = job_n_3_1
    
    i = i + 1
    if(i % 1000 == 0):
        jobs_whole.to_csv(outfile)
        
    #    break

#large_jobs['real_pow'] = real_pow
#large_jobs['runned_alone'] = runned_alone
#large_jobs['pow_quality'] = quality
# write
jobs_whole.to_csv(outfile)



In [44]:
#large_jobs.to_csv(outfile)

In [49]:
#large_jobs['real_pow'] = real_pow
#large_jobs['runned_alone'] = runned_alone
#large_jobs['pow_quality'] = quality
# write
#outfile = datadir + 'CPUs/' + interval_comment + "/large_jobs_real_pow_3.csv"
#large_jobs.to_csv(outfile)

In [43]:
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "01" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_01 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "02" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_02 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "03" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_03 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "04" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_04 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "05" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_05 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "06" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_06 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "07" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_07 = pd.read_csv(infile_node)

infile_node = datadir + "CPUs/" + interval_comment + "/node" + "08" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_08 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "09" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_09 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "10" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_10 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "11" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_11 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "12" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_12 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "13" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_13 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "14" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_14 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "15" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_15= pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "16" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_16 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "17" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_17 = pd.read_csv(infile_node)

In [44]:
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "18" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_18 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "19" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_19 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "20" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_20 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "21" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_21 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "22" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_22 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "23" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_23 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "24" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_24 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "25" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_25 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "26" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_26 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "27" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_27 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "28" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_28 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "29" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_29 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "30" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_30 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "31" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_31 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "32" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_32 = pd.read_csv(infile_node)

In [45]:
# most used nodes

infile_node = datadir + "CPUs/" + interval_comment + "/node" + "33" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_33 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "34" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_34 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "35" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_35 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "36" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_36 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "37" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_37 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "38" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_38 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "39" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_39 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "40" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_40 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "41" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_41 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "42" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_42 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "44" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_44 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "45" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_45 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "46" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_46 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "47" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_47 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "48" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_48 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "49" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_49 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "50" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_50 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "51" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_51 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "52" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_52 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "53" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_53 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "54" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_54 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "55" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_55 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "56" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_56 = pd.read_csv(infile_node)

In [46]:
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "57" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_57 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "58" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_58 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "59" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_59 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "60" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_60 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "61" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_61 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "62" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_62 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "63" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_63 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "64" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_64 = pd.read_csv(infile_node)



In [47]:
### TODO I probably have to load more than one node measurements file
def calculate_job_consumption_2(job_id_string):
    
    job_real_pow_consumption = 0
    job_runned_alone = True
    job_pow_quality = 1.0
    job_good_nodes = 0 # node where we have at most 1 measurement in the interval
    job_measurements_tot = 0
    job_measurements_missing_tot = 0
    
    partial_pow_consumption = 0
    
    job_to_nodes = merged_jobs_to_nodes_long[merged_jobs_to_nodes_long['job_id_string'] == job_id_string]
    job_row = job_to_nodes.iloc[0]
    job_to_nodes_unique = job_to_nodes.node_id.unique() # series with the nodes id
    ### print some jobs info
    print("job {} use {} cores in this node".format(job_id_string, str(job_row['ncpus'])))
    print("job {} start at {} and end at {}".format(job_id_string, str(job_row['run_start_time']), str(job_row['end_time'])))
    print("job {} requires {} nodes and {} cores".format(job_id_string, str(job_row['node_req']), str(job_row['cpu_req'])))
    print(job_to_nodes_unique)
    
    job_n_2_1 = 0
    job_n_2_2 = 0
    job_n_3_1 = 0
    
    for index, row in job_to_nodes.iterrows():
        
        node_type = nodes_data.iloc[row['node_id']-1]['cpu_type']
        if(node_type == '2_1_ghz'):
            job_n_2_1 += 1
        elif(node_type == '2_2_ghz'):
            job_n_2_2 += 1
        elif(node_type == '3_1_ghz'):
            job_n_3_1 += 1
        
        node_idle_core = nodes_data.iloc[row['node_id']-1]['core_idle']
        
        if(row['node_id'] == 1):
            node_measurements = node_measurements_01
        elif(row['node_id'] == 2):
            node_measurements = node_measurements_02
        elif(row['node_id'] == 3):
            node_measurements = node_measurements_03
        elif(row['node_id'] == 4):
            node_measurements = node_measurements_04
        elif(row['node_id'] == 5):
            node_measurements = node_measurements_05
        elif(row['node_id'] == 6):
            node_measurements = node_measurements_06
        elif(row['node_id'] == 7):
            node_measurements = node_measurements_07
        elif(row['node_id'] == 8):
            node_measurements = node_measurements_08
        elif(row['node_id'] == 9):
            node_measurements = node_measurements_09
        elif(row['node_id'] == 10):
            node_measurements = node_measurements_10
        elif(row['node_id'] == 11):
            node_measurements = node_measurements_11
        elif(row['node_id'] == 12):
            node_measurements = node_measurements_12
        elif(row['node_id'] == 13):
            node_measurements = node_measurements_13
        elif(row['node_id'] == 14):
            node_measurements = node_measurements_14
        elif(row['node_id'] == 15):
            node_measurements = node_measurements_15
        elif(row['node_id'] == 16):
            node_measurements = node_measurements_16
        elif(row['node_id'] == 17):
            node_measurements = node_measurements_17
        elif(row['node_id'] == 18):
            node_measurements = node_measurements_18
        elif(row['node_id'] == 19):
            node_measurements = node_measurements_19
        elif(row['node_id'] == 20):
            node_measurements = node_measurements_20
        elif(row['node_id'] == 21):
            node_measurements = node_measurements_21
        elif(row['node_id'] == 22):
            node_measurements = node_measurements_22
        elif(row['node_id'] == 23):
            node_measurements = node_measurements_23
        elif(row['node_id'] == 24):
            node_measurements = node_measurements_24
        elif(row['node_id'] == 25):
            node_measurements = node_measurements_25
        elif(row['node_id'] == 26):
            node_measurements = node_measurements_26
        elif(row['node_id'] == 27):
            node_measurements = node_measurements_27
        elif(row['node_id'] == 28):
            node_measurements = node_measurements_28
        elif(row['node_id'] == 29):
            node_measurements = node_measurements_29
        elif(row['node_id'] == 30):
            node_measurements = node_measurements_30
        elif(row['node_id'] == 31):
            node_measurements = node_measurements_31
        elif(row['node_id'] == 32):
            node_measurements = node_measurements_32
        elif(row['node_id'] == 33):
            node_measurements = node_measurements_33
        elif(row['node_id'] == 34):
            node_measurements = node_measurements_34
        elif(row['node_id'] == 35):
            node_measurements = node_measurements_35
        elif(row['node_id'] == 36):
            node_measurements = node_measurements_36
        elif(row['node_id'] == 37):
            node_measurements = node_measurements_37
        elif(row['node_id'] == 38):
            node_measurements = node_measurements_38
        elif(row['node_id'] == 39):
            node_measurements = node_measurements_39
        elif(row['node_id'] == 40):
            node_measurements = node_measurements_40
        elif(row['node_id'] == 41):
            node_measurements = node_measurements_41
        elif(row['node_id'] == 42):
            node_measurements = node_measurements_42
        elif(row['node_id'] == 44):
            node_measurements = node_measurements_44
        elif(row['node_id'] == 45):
            node_measurements = node_measurements_45
        elif(row['node_id'] == 46):
            node_measurements = node_measurements_46
        elif(row['node_id'] == 47):
            node_measurements = node_measurements_47
        elif(row['node_id'] == 48):
            node_measurements = node_measurements_48
        elif(row['node_id'] == 49):
            node_measurements = node_measurements_49
        elif(row['node_id'] == 50):
            node_measurements = node_measurements_50
        elif(row['node_id'] == 51):
            node_measurements = node_measurements_51
        elif(row['node_id'] == 52):
            node_measurements = node_measurements_52
        elif(row['node_id'] == 53):
            node_measurements = node_measurements_53
        elif(row['node_id'] == 54):
            node_measurements = node_measurements_54
        elif(row['node_id'] == 55):
            node_measurements = node_measurements_55
        elif(row['node_id'] == 56):
            node_measurements = node_measurements_56
        elif(row['node_id'] == 57):
            node_measurements = node_measurements_57
        elif(row['node_id'] == 58):
            node_measurements = node_measurements_58
        elif(row['node_id'] == 59):
            node_measurements = node_measurements_59
        elif(row['node_id'] == 60):
            node_measurements = node_measurements_60
        elif(row['node_id'] == 61):
            node_measurements = node_measurements_61
        elif(row['node_id'] == 62):
            node_measurements = node_measurements_62
        elif(row['node_id'] == 63):
            node_measurements = node_measurements_63
        elif(row['node_id'] == 64):
            node_measurements = node_measurements_64
        
        
        else:
            ### add a 0 to the number if is less than 10 ("1" -> "01", ...)
            node_str = "0" + str(row['node_id']) if row['node_id'] < 10 else str(row['node_id'])
            print("node {}".format(node_str))
            ### open the measurements node file I/O
            infile_node = datadir + "CPUs/" + interval_comment + "/node" + node_str + suffix + interval_comment + "_active_cores_and_jobs.csv"
            node_measurements = pd.read_csv(infile_node)
    
        ### take only the entries of the interval where the job was running
        interval_measurements = node_measurements.loc[(pd.to_datetime(node_measurements['timestamp']) >= pd.to_datetime(job_row['run_start_time'])) & (pd.to_datetime(node_measurements['timestamp']) <= pd.to_datetime((job_row['end_time'])) - np.timedelta64(5, 's'))]
        
        ### TODO drop missing measurements (calculate the percentage)
        n_node_measurements = interval_measurements.shape[0]
        job_measurements_tot += n_node_measurements
        
        interval_measurements = interval_measurements.dropna()
        # print("interval_measurements {}".format(interval_measurements.shape[0]))
        print("{}/{} intervals missing".format(n_node_measurements-interval_measurements.shape[0], n_node_measurements))
        job_measurements_missing_tot += n_node_measurements-interval_measurements.shape[0]
        
        # print(interval_measurements)
        ### drop where active cores is greater than 16 - only on node 30
        interval_measurements = interval_measurements[interval_measurements['active_cores'] <= 16]
        ### TODO drop row with 0 active_cores (or less than the current job used cores)
        interval_measurements = interval_measurements[interval_measurements['active_cores'] >= row['ncpus']]  
        
        if(interval_measurements.shape[0] != 0):
            job_good_nodes += 1
        
        ### group the intervals wehere the partial_pow_cons can be obtaied with the same instance of the formula
        ### take the mean for the pow columns
        # interval_grouped = interval_measurements.reset_index().groupby(["active_cores", "active_jobs", "active_gpus", "active_mics"]).mean()
        interval_grouped = interval_measurements.groupby(["active_cores", "active_jobs", "active_gpus", "active_mics"])
        counts = interval_grouped.size().to_frame(name='counts')
        interval_grouped = (counts
         .join(interval_grouped.agg({'pow_tot':'mean'}).rename(columns={'pow_tot': 'pow_tot_mean'}))
         .reset_index()
        )
        interval_grouped.sort_values('active_jobs')
        
        if(interval_grouped.shape[0]>1):
            job_runned_alone = False
        
        ### apply the formula on each grouped data
        #for index_group, row_group in interval_grouped.iterrows():
            # n_active_cores = index_group[0]
        #    n_active_cores = row_group['active_cores']
        #    partial_pow_consumption += (row_group['pow_tot_0_mean'] + row_group['pow_tot_1_mean']) * row['ncpus'] / n_active_cores
        
        if(interval_grouped.shape[0] != 0):
            #print(interval_grouped)
            #interval_grouped['pow_tot'] = (interval_grouped['pow_tot_0_mean'] + interval_grouped['pow_tot_1_mean']) * row['ncpus'] / interval_grouped['active_cores'] 
            interval_grouped['pow_tot'] = (interval_grouped['pow_tot_mean'] - (16 - interval_grouped['active_cores'])*node_idle_core) * row['ncpus'] / interval_grouped['active_cores'] 
            
            # not_alone_counts = interval_grouped['counts'].loc[interval_grouped['active_jobs'] != 1].sum()
            # interval_grouped.loc[interval_grouped['active_jobs'] == 1, ['counts']] += not_alone_counts
            print(interval_grouped)
            partial_pow_consumption = np.average(interval_grouped['pow_tot'], weights=interval_grouped['counts'])
            # partial_pow_consumption = partial_pow_consumption / interval_grouped.shape[0]
            # print(interval_grouped)
            
        print("partial measurement: {}".format(partial_pow_consumption))
        job_real_pow_consumption += partial_pow_consumption
        partial_pow_consumption = 0
    
    print("{}/{} node measurements are good".format(job_good_nodes, job_to_nodes_unique.shape[0]))
  
    # print("job_real_pow_consumption is {}".format(job_real_pow_consumption))
    ### TODO if some entire node data is missing, approximate the amount to add
    if(job_good_nodes != 0):
        job_real_pow_consumption = job_real_pow_consumption + (job_real_pow_consumption / job_good_nodes) * (job_to_nodes_unique.shape[0] - job_good_nodes)
    else:
        job_real_pow_consumption = 8.75 * row['cpu_req']
        job_pow_quality = 0.0
    print(job_real_pow_consumption)  
    
    if(job_measurements_missing_tot != 0):
        job_pow_quality = 1 - (job_measurements_missing_tot / job_measurements_tot)
        
    return job_real_pow_consumption, job_runned_alone, job_pow_quality, job_n_2_1, job_n_2_2, job_n_3_1

In [ ]:
#long_jobs = long_jobs.drop(['real_pow', 'runned_alone', 'real_pow_quality'], axis=1)

short_jobs['real_pow'] = 0.0
short_jobs['runned_alone'] = True
short_jobs['real_pow_quality'] = 0.0

short_jobs['n_2_1'] = 0
short_jobs['n_2_2'] = 0
short_jobs['n_3_1'] = 0

In [33]:
infile = datadir + 'CPUs/' + interval_comment + "/short_jobs_real_pow.csv"
short_jobs_to_compute = pd.read_csv(infile, index_col=0)
print(short_jobs_to_compute.shape[0])

short_jobs_to_compute = short_jobs_to_compute[short_jobs_to_compute['real_pow'] == 0.0]
print(short_jobs_to_compute.shape[0])

206634
94545


### compute the real power consumption - short jobs

In [2]:
### remove to continue from a specific job



i = 0
outfile = datadir + 'CPUs/' + interval_comment + "/short_jobs_real_pow.csv"
for index, row in short_jobs_to_compute.iterrows():
#for index, row in large_jobs_tail.iterrows():
    print()
    print("{}/{}".format(i, short_jobs_to_compute.shape[0]))

    #real_pow[i], runned_alone[i], quality[i] = calculate_job_consumption(row['job_id_string'])
    real_pow_val, runned_alone_val, quality_val, job_n_2_1, job_n_2_2, job_n_3_1 = calculate_job_consumption_short(row['job_id_string'])
    short_jobs.at[index,'real_pow'] = real_pow_val
    short_jobs.at[index,'runned_alone'] = runned_alone_val
    short_jobs.at[index,'real_pow_quality'] = quality_val
    
    short_jobs.at[index,'n_2_1'] = job_n_2_1
    short_jobs.at[index,'n_2_2'] = job_n_2_2
    short_jobs.at[index,'n_3_1'] = job_n_3_1
    
    i = i + 1
    if(i % 1000 == 0):
        short_jobs.to_csv(outfile)
        
    #    break

#large_jobs['real_pow'] = real_pow
#large_jobs['runned_alone'] = runned_alone
#large_jobs['pow_quality'] = quality
# write
short_jobs.to_csv(outfile)

NameError: name 'datadir' is not defined

In [35]:
### TODO I probably have to load more than one node measurements file
def calculate_job_consumption_short(job_id_string):
    
    job_real_pow_consumption = 0
    job_runned_alone = True
    job_pow_quality = 1.0
    job_good_nodes = 0 # node where we have at most 1 measurement in the interval
    job_measurements_tot = 0
    job_measurements_missing_tot = 0
    
    partial_pow_consumption = 0
    
    job_to_nodes = merged_jobs_to_nodes_short[merged_jobs_to_nodes_short['job_id_string'] == job_id_string]
    job_row = job_to_nodes.iloc[0]
    job_to_nodes_unique = job_to_nodes.node_id.unique() # series with the nodes id
    ### print some jobs info
    print("job {} use {} cores in this node".format(job_id_string, str(job_row['ncpus'])))
    #print("job {} start at {} and end at {}".format(job_id_string, str(job_row['run_start_time']), str(job_row['end_time'])))
    print("job {} requires {} nodes and {} cores".format(job_id_string, str(job_row['node_req']), str(job_row['cpu_req'])))
    print(job_to_nodes_unique)
    
    job_n_2_1 = 0
    job_n_2_2 = 0
    job_n_3_1 = 0
    
    for index, row in job_to_nodes.iterrows():
        
        node_type = nodes_data.iloc[row['node_id']-1]['cpu_type']
        if(node_type == '2_1_ghz'):
            job_n_2_1 += 1
        elif(node_type == '2_2_ghz'):
            job_n_2_2 += 1
        elif(node_type == '3_1_ghz'):
            job_n_3_1 += 1
        
        node_idle_core = nodes_data.iloc[row['node_id']-1]['core_idle']
        
        if(row['node_id'] == 1):
            node_measurements = node_measurements_01
        elif(row['node_id'] == 2):
            node_measurements = node_measurements_02
        elif(row['node_id'] == 3):
            node_measurements = node_measurements_03
        elif(row['node_id'] == 4):
            node_measurements = node_measurements_04
        elif(row['node_id'] == 5):
            node_measurements = node_measurements_05
        elif(row['node_id'] == 6):
            node_measurements = node_measurements_06
        elif(row['node_id'] == 7):
            node_measurements = node_measurements_07
        elif(row['node_id'] == 8):
            node_measurements = node_measurements_08
        elif(row['node_id'] == 9):
            node_measurements = node_measurements_09
        elif(row['node_id'] == 10):
            node_measurements = node_measurements_10
        elif(row['node_id'] == 11):
            node_measurements = node_measurements_11
        elif(row['node_id'] == 12):
            node_measurements = node_measurements_12
        elif(row['node_id'] == 13):
            node_measurements = node_measurements_13
        elif(row['node_id'] == 14):
            node_measurements = node_measurements_14
        elif(row['node_id'] == 15):
            node_measurements = node_measurements_15
        elif(row['node_id'] == 16):
            node_measurements = node_measurements_16
        elif(row['node_id'] == 17):
            node_measurements = node_measurements_17
        elif(row['node_id'] == 18):
            node_measurements = node_measurements_18
        elif(row['node_id'] == 19):
            node_measurements = node_measurements_19
        elif(row['node_id'] == 20):
            node_measurements = node_measurements_20
        elif(row['node_id'] == 21):
            node_measurements = node_measurements_21
        elif(row['node_id'] == 22):
            node_measurements = node_measurements_22
        elif(row['node_id'] == 23):
            node_measurements = node_measurements_23
        elif(row['node_id'] == 24):
            node_measurements = node_measurements_24
        elif(row['node_id'] == 25):
            node_measurements = node_measurements_25
        elif(row['node_id'] == 26):
            node_measurements = node_measurements_26
        elif(row['node_id'] == 27):
            node_measurements = node_measurements_27
        elif(row['node_id'] == 28):
            node_measurements = node_measurements_28
        elif(row['node_id'] == 29):
            node_measurements = node_measurements_29
        elif(row['node_id'] == 30):
            node_measurements = node_measurements_30
        elif(row['node_id'] == 31):
            node_measurements = node_measurements_31
        elif(row['node_id'] == 32):
            node_measurements = node_measurements_32
        elif(row['node_id'] == 33):
            node_measurements = node_measurements_33
        elif(row['node_id'] == 34):
            node_measurements = node_measurements_34
        elif(row['node_id'] == 35):
            node_measurements = node_measurements_35
        elif(row['node_id'] == 36):
            node_measurements = node_measurements_36
        elif(row['node_id'] == 37):
            node_measurements = node_measurements_37
        elif(row['node_id'] == 38):
            node_measurements = node_measurements_38
        elif(row['node_id'] == 39):
            node_measurements = node_measurements_39
        elif(row['node_id'] == 40):
            node_measurements = node_measurements_40
        elif(row['node_id'] == 41):
            node_measurements = node_measurements_41
        elif(row['node_id'] == 42):
            node_measurements = node_measurements_42
        elif(row['node_id'] == 44):
            node_measurements = node_measurements_44
        elif(row['node_id'] == 45):
            node_measurements = node_measurements_45
        elif(row['node_id'] == 46):
            node_measurements = node_measurements_46
        elif(row['node_id'] == 47):
            node_measurements = node_measurements_47
        elif(row['node_id'] == 48):
            node_measurements = node_measurements_48
        elif(row['node_id'] == 49):
            node_measurements = node_measurements_49
        elif(row['node_id'] == 50):
            node_measurements = node_measurements_50
        elif(row['node_id'] == 51):
            node_measurements = node_measurements_51
        elif(row['node_id'] == 52):
            node_measurements = node_measurements_52
        elif(row['node_id'] == 53):
            node_measurements = node_measurements_53
        elif(row['node_id'] == 54):
            node_measurements = node_measurements_54
        elif(row['node_id'] == 55):
            node_measurements = node_measurements_55
        elif(row['node_id'] == 56):
            node_measurements = node_measurements_56
        elif(row['node_id'] == 57):
            node_measurements = node_measurements_57
        elif(row['node_id'] == 58):
            node_measurements = node_measurements_58
        elif(row['node_id'] == 59):
            node_measurements = node_measurements_59
        elif(row['node_id'] == 60):
            node_measurements = node_measurements_60
        elif(row['node_id'] == 61):
            node_measurements = node_measurements_61
        elif(row['node_id'] == 62):
            node_measurements = node_measurements_62
        elif(row['node_id'] == 63):
            node_measurements = node_measurements_63
        elif(row['node_id'] == 64):
            node_measurements = node_measurements_64
        
        
        else:
            ### add a 0 to the number if is less than 10 ("1" -> "01", ...)
            node_str = "0" + str(row['node_id']) if row['node_id'] < 10 else str(row['node_id'])
            print("node {}".format(node_str))
            ### open the measurements node file I/O
            infile_node = datadir + "CPUs/" + interval_comment + "/node" + node_str + suffix + interval_comment + "_active_cores_and_jobs.csv"
            node_measurements = pd.read_csv(infile_node)
    
    
        measure = node_measurements[node_measurements['timestamp'] == job_row['run_start_time'][:-3]+':00']
        
        if(measure.shape[0] != 0):
            if(measure['active_cores'].iloc[0] != 0):
                job_runned_alone = False
            
            active_cores = measure['active_cores'].iloc[0] + row['ncpus']
            if(active_cores > 16):
                active_cores = 16
            job_real_pow_consumption = job_real_pow_consumption + (measure['pow_tot_0'].iloc[0] + measure['pow_tot_1'].iloc[0] - (16 - active_cores) * node_idle_core) * row['ncpus'] / active_cores
        else:
            print("error measure")
            job_pow_quality = 0.0
    
    if(job_real_pow_consumption < 0):
        job_real_pow_consumption = 0
        job_pow_quality = 0.0
    print(job_real_pow_consumption)
        
    return job_real_pow_consumption, job_runned_alone, job_pow_quality, job_n_2_1, job_n_2_2, job_n_3_1

In [57]:
#### calculate the power consumption of a specifified job



job_id_string = "498462.node129" # non ha misurazioni
job_id_string = "498463.node129" # non ha misurazioni
job_id_string = "498464.node129" # non ha misurazioni
job_id_string = "498465.node129" # [ 35]

job_id_string = "498699.node129" # [ 37]
job_id_string = "498458.node129" # non ha misurazioni
job_id_string = "498458.node129" # non ha misurazioni
job_id_string = "498461.node129" # single node 16 cores [ 27]
job_id_string = "500752.node129" # 2 nodi, 24 cores [ 16 20] 20 totalmente assente
job_id_string = "500899.node129" # [ 15 23 32 28 25 19]

job_id_string = "498467.node129"

job_id_string = "498460.node129" # 6 nodi, 96 cores [ 9 28 25 19 26 29]
job_id_string = "498466.node129" # [ 36 38 40 42 50 56 59 61 62 63 54]
job_id_string = "500752.node129" # 2 nodi, 24 cores [ 16 20] 20 totalmente assente
job_id_string = "498459.node129" # single node 16 cores [ 9]
job_id_string = "498699.node129" # [ 37]
job_id_string = "498458.node129" # [ ]
job_id_string = "498460.node129" # 6 nodi, 96 cores [ 9 28 25 19 26 29]
job_id_string = "500899.node129" # [ 15 23 32 28 25 19]


print("job_id_string: {}".format(job_id_string))



job_id_string: 500899.node129


In [63]:
calculate_job_consumption_2(job_id_string)

job 500899.node129 use 16 cores in this node
job 500899.node129 start at 2014-03-31 19:24:06 and end at 2014-03-31 19:40:33
job 500899.node129 requires 6 nodes and 96 cores
[15 23 32 28 25 19]
node 15
0/15 intervals missing
   active_cores  active_jobs  active_gpus  active_mics  counts  pow_tot_0_mean  pow_tot_1_mean
0            16            1            0            0      15       94.476258         96.9878
partial measurement: 191.46405847759212
0/15 intervals missing
   active_cores  active_jobs  active_gpus  active_mics  counts  pow_tot_0_mean  pow_tot_1_mean
0            16            1            0            0      15       93.407682       92.983533
partial measurement: 186.3912153826789
node 32
15/15 intervals missing
partial measurement: 0
0/15 intervals missing
   active_cores  active_jobs  active_gpus  active_mics  counts  pow_tot_0_mean  pow_tot_1_mean
0            16            1            0            0      15       97.068548       98.201442
partial measurement: 195.2

(1147.5136811412594, True, 0.8333333333333334, 4, 2, 0)